In [85]:
import pandas as pd
import numpy as np

In [86]:
population = pd.read_csv('input_file_v1_dashboard.csv')
ward = pd.read_csv('Facilities_in_Mumbai_COVID_19_Cases.csv')

In [87]:
population.head()

,Country,State,City,Ward location,Ward,Population,Density per square kilometer,Senior citizen %,Area,Educational institutes,Hospitals,Supermarts/malls,Critical congestion points,Commercial complexes,Residential areas
0,India,Maharashtra,Mumbai,Colaba,A,210847,16868.0,0.14,12.50,54,40,20,0,0,0
1,India,Maharashtra,Mumbai,Sanhurst Road,B,140633,56936.0,0.14,2.47,48,32,32,0,0,0
2,India,Maharashtra,Mumbai,Marine Lines,C,202922,114001.0,0.14,1.78,43,23,21,0,0,0
3,India,Maharashtra,Mumbai,Grant Road,D,382841,57744.0,0.14,6.63,23,43,21,0,0,0
4,India,Maharashtra,Mumbai,Byculla,E,440335,59505.0,0.14,7.40,32,57,12,0,0,0


In [88]:
ward.head()

,Ward,Number of Cases- Very Congested Area,Number of Cases- Medium Congested,Number of Cases- Standalone Structure,Date Entered
0,NaN,NaN,NaN,NaN,11th April or Earlier
1,A,1.0,NaN,5.0,NaN
2,B,3.0,NaN,12.0,NaN
3,C,3.0,NaN,9.0,NaN
4,D,3.0,3.0,18.0,NaN


In [89]:
ward = ward.drop(0).reset_index(drop=True)
ward['Number of Cases- Medium Congested'].fillna(-1, inplace=True)

In [90]:
cols = ['Ward','Ward location','Number of Cases- Very Congested Area','Number of Cases- Medium Congested',
        'Number of Cases- Standalone Structure','Population']
df = pd.merge(ward, population)[cols]

In [91]:
rename_cols = {"Number of Cases- Very Congested Area":"VeryCongested",
               "Number of Cases- Medium Congested":"MediumCongested",
               "Number of Cases- Standalone Structure":"Standalone",
              "Ward location":"Loc"}
df = df.rename(rename_cols, axis=1)
df.head()

,Ward,Loc,VeryCongested,MediumCongested,Standalone,Population
0,A,Colaba,1.0,-1.0,5.0,210847
1,B,Sanhurst Road,3.0,-1.0,12.0,140633
2,C,Marine Lines,3.0,-1.0,9.0,202922
3,D,Grant Road,3.0,3.0,18.0,382841
4,E,Byculla,2.0,3.0,5.0,440335


In [92]:
condition = (df.MediumCongested == 0) | (df.VeryCongested == 0) | (df.Standalone == 0)

df['Color_Range'] = (df.Standalone*1.5+df.MediumCongested*2.6+df.VeryCongested*3.6)/df.Population

df['Color'] = np.where(condition, "Green", np.where(df.Color_Range >= 0.0002 , "Red", df.Standalone))
df['Color'] = np.where(condition, "Green", np.where((df.Color_Range >= 0.0001) &  (df.Color_Range < 0.0002), "Orange", df.Color))
df['Color'] = np.where(condition, "Green", np.where((df.Color_Range > 0) & (df.Color_Range < 0.0001), "Blue", df.Color))

In [93]:
df['Color'].value_counts()

Orange    9
Green     7
Blue      7
Red       1
Name: Color, dtype: int64

In [94]:
df

,Ward,Loc,VeryCongested,MediumCongested,Standalone,Population,Color_Range,Color
0,A,Colaba,1.0,-1.0,5.0,210847,0.000040,Blue
1,B,Sanhurst Road,3.0,-1.0,12.0,140633,0.000186,Orange
2,C,Marine Lines,3.0,-1.0,9.0,202922,0.000107,Orange
3,D,Grant Road,3.0,3.0,18.0,382841,0.000119,Orange
4,E,Byculla,2.0,3.0,5.0,440335,0.000051,Blue
5,F/N,Matunga,5.0,0.0,10.0,524393,0.000063,Green
6,F/S,Parel,1.0,2.0,11.0,396122,0.000064,Blue
7,G/S,Elphinstone,24.0,3.0,28.0,457931,0.000297,Red
8,G/N,Dadar/Plaza,6.0,0.0,12.0,582007,0.000068,Green
9,H/E,Khar/Santacruz,11.0,1.0,16.0,580835,0.000114,Orange


In [95]:
df.to_csv('Color.csv', index=False, header=True)